In [1]:
import cv2
import mysql.connector
from datetime import datetime
from deepface import DeepFace
import os
import pickle

class Recognition:
    def __init__(self, db, session_id, course_id):
        self.db = db
        self.session_id = session_id
        self.course_id = course_id
        self.confidence_threshold = 0.6
        self.load_encodings()

    def load_encodings(self):
        # Load saved embeddings and file paths
        if os.path.exists('known_encodings.pkl'):
            with open('known_encodings.pkl', 'rb') as f:
                self.results = pickle.load(f)
                self.known_encodings = [result[1] for result in self.results]
                self.file_paths = [result[0] for result in self.results]
        else:
            self.results = []
            self.known_encodings = []
            self.file_paths = []

    def retrain(self):
        def process_folder(student_folder):
            student_id = os.path.basename(student_folder)
            image_files = [os.path.join(student_folder, file) for file in os.listdir(student_folder) if file.lower().endswith(('.png', '.jpg', '.jpeg'))]
            encodings = []
            for img_path in image_files:
                img = cv2.imread(img_path)
                encoding = DeepFace.represent(img, model_name='VGG-Face', enforce_detection=False)
                if encoding:
                    encodings.append((img_path, encoding[0]['embedding']))
            return encodings

        db_folder = 'db'
        student_folders = [os.path.join(db_folder, folder) for folder in os.listdir(db_folder) if os.path.isdir(os.path.join(db_folder, folder))]

        self.results = []
        for student_folder in student_folders:
            self.results.extend(process_folder(student_folder))

        # Save computed embeddings and corresponding file paths
        with open('known_encodings.pkl', 'wb') as f:
            pickle.dump(self.results, f)

        self.load_encodings()

    def mark_attendance(self, student_id):
        """Marks attendance for a recognized student in the database."""
        try:
            cursor = self.db.cursor()
            # Fetch student's name from the database based on the student_id
            sql = "SELECT fullname FROM student_details WHERE student_id = %s"
            val = (student_id,)
            cursor.execute(sql, val)
            result = cursor.fetchone()
            if result:
                student_name = result[0]
                attendance_status = f"Attendance taken for student {student_name}"
            else:
                attendance_status = "Attendance not taken"
            # Check if the student has already been marked present for the given course and session
            sql = "SELECT * FROM attendance WHERE student_id = %s AND course_id = %s AND session_id = %s"
            val = (student_id, self.course_id, self.session_id)
            cursor.execute(sql, val)
            result = cursor.fetchone()
            if result:
                print(f"Student with ID {student_id} has already been marked present for course {self.course_id} and session {self.session_id}. Skipping insertion.")
                return
            else:
                sql = "INSERT INTO attendance (course_id, session_id, timestamp, status, student_id) VALUES (%s, %s, %s, %s, %s)"
                val = (self.course_id, self.session_id, datetime.now(), "Present", student_id)
                cursor.execute(sql, val)
                self.db.commit()
                cursor.close()
                print(f"Attendance marked for student with ID {student_id} for course {self.course_id} and session {self.session_id}.")
        except mysql.connector.Error as err:
            print(f"Database error: {err}")

    def retrain(self):
        self.retrain()

    def recognize(self):
        cap = cv2.VideoCapture(0)
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

        while True:
            success, img = cap.read()
            if not success:
                print("Error reading frame from camera.")
                break

            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray, 1.3, 5)

            for (x, y, w, h) in faces:
                face_img = img[y:y+h, x:x+w]

                face_encoding = DeepFace.represent(face_img, model_name='VGG-Face', enforce_detection=False)

                if face_encoding:
                    face_encoding = face_encoding[0]['embedding']

                    best_match_id = None
                    min_distance = float('inf')

                    for j, known_encoding in enumerate(self.known_encodings):
                        if len(known_encoding) == len(face_encoding):
                            result = DeepFace.verify(known_encoding, face_encoding, model_name='VGG-Face', silent=True)
                            if result['verified'] and result['distance'] < min_distance:
                                if result['distance'] > self.confidence_threshold:
                                    best_match_id = os.path.splitext(os.path.basename(self.file_paths[j]))[0]
                                    min_distance = result['distance']

                    # Fetch student's name from the database based on student ID
                    student_name = None
                    if best_match_id:
                        cursor = self.db.cursor()
                        sql = "SELECT fullname FROM student_details WHERE student_id = %s"
                        val = (best_match_id,)
                        cursor.execute(sql, val)
                        result = cursor.fetchone()
                        if result:
                            student_name = result[0]

                    # Check if the student is recognized and mark attendance accordingly
                    if best_match_id:
                        self.mark_attendance(best_match_id)
                        if student_name:
                            attendance_status = f"Attendance taken for student {student_name}"
                        else:
                            attendance_status = "Attendance taken for student"
                    else:
                        attendance_status = "Attendance not taken"

                    # Draw rectangle and label with attendance status and student's name
                    color = (0, 255, 0) if best_match_id else (0, 0, 255)
                    cv2.rectangle(img, (x, y), (x+w, y+h), color, 2)
                    cv2.putText(img, attendance_status, (x, y-10), cv2.FONT_ITALIC, 0.9, color, 2)
                    if best_match_id and student_name:
                        cv2.putText(img, student_name, (x, y+h+20), cv2.FONT_ITALIC, 0.9, color, 2)

            cv2.imshow("Faces with Rectangles", img)

            if cv2.waitKey(1) == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()

In [5]:
import cv2
import mysql.connector
from datetime import datetime
from deepface import DeepFace
import os
import pickle

class Recognition:
    def __init__(self, db, session_id, course_id):
        self.db = db
        self.session_id = session_id
        self.course_id = course_id
        self.confidence_threshold = 0.6
        self.load_encodings()

    def load_encodings(self):
        # Load saved embeddings and file paths
        if os.path.exists('known_encodings.pkl'):
            with open('known_encodings.pkl', 'rb') as f:
                self.results = pickle.load(f)
                self.known_encodings = [result[1] for result in self.results]
                self.file_paths = [result[0] for result in self.results]
        else:
            self.results = []
            self.known_encodings = []
            self.file_paths = []

    def retrain(self):
        def process_folder(student_folder):
            student_id = os.path.basename(student_folder)
            image_files = [os.path.join(student_folder, file) for file in os.listdir(student_folder) if file.lower().endswith(('.png', '.jpg', '.jpeg'))]
            encodings = []
            for img_path in image_files:
                img = cv2.imread(img_path)
                encoding = DeepFace.represent(img, model_name='VGG-Face', enforce_detection=False)
                if encoding:
                    encodings.append((img_path, encoding[0]['embedding']))
            return encodings

        db_folder = 'db'
        student_folders = [os.path.join(db_folder, folder) for folder in os.listdir(db_folder) if os.path.isdir(os.path.join(db_folder, folder))]

        self.results = []
        for student_folder in student_folders:
            self.results.extend(process_folder(student_folder))

        # Save computed embeddings and corresponding file paths
        with open('known_encodings.pkl', 'wb') as f:
            pickle.dump(self.results, f)

        self.load_encodings()

    def mark_attendance(self, student_id):
        """Marks attendance for a recognized student in the database."""
        try:
            cursor = self.db.cursor()
            # Fetch student's name from the database based on the student_id
            sql = "SELECT fullname FROM student_details WHERE student_id = %s"
            val = (student_id,)
            cursor.execute(sql, val)
            result = cursor.fetchone()
            if result:
                student_name = result[0]
                attendance_status = f"Attendance taken for student {student_name}"
            else:
                attendance_status = "Attendance not taken"
            # Check if the student has already been marked present for the given course and session
            sql = "SELECT * FROM attendance WHERE student_id = %s AND course_id = %s AND session_id = %s"
            val = (student_id, self.course_id, self.session_id)
            cursor.execute(sql, val)
            result = cursor.fetchone()
            if result:
                print(f"Student with ID {student_id} has already been marked present for course {self.course_id} and session {self.session_id}. Skipping insertion.")
                return
            else:
                sql = "INSERT INTO attendance (course_id, session_id, timestamp, status, student_id) VALUES (%s, %s, %s, %s, %s)"
                val = (self.course_id, self.session_id, datetime.now(), "Present", student_id)
                cursor.execute(sql, val)
                self.db.commit()
                cursor.close()
                print(f"Attendance marked for student with ID {student_id} for course {self.course_id} and session {self.session_id}.")
        except mysql.connector.Error as err:
            print(f"Database error: {err}")

    def retrain(self):
        self.retrain()

    def recognize(self):
        cap = cv2.VideoCapture(0)
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

        while True:
            success, img = cap.read()
            if not success:
                print("Error reading frame from camera.")
                break

            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray, 1.3, 5)

            for (x, y, w, h) in faces:
                face_img = img[y:y+h, x:x+w]

                face_encoding = DeepFace.represent(face_img, model_name='VGG-Face', enforce_detection=False)

                if face_encoding:
                    face_encoding = face_encoding[0]['embedding']

                    best_match_id = None
                    min_distance = float('inf')

                    for j, known_encoding in enumerate(self.known_encodings):
                        if len(known_encoding) == len(face_encoding):
                            result = DeepFace.verify(known_encoding, face_encoding, model_name='VGG-Face', silent=True)
                            if result['verified'] and result['distance'] < min_distance:
                                if result['distance'] > self.confidence_threshold:
                                    best_match_id = os.path.splitext(os.path.basename(self.file_paths[j]))[0]
                                    min_distance = result['distance']

                    # Fetch student's name from the database based on student ID
                    student_name = None
                    if best_match_id:
                        cursor = self.db.cursor()
                        sql = "SELECT fullname FROM student_details WHERE student_id = %s"
                        val = (best_match_id,)
                        cursor.execute(sql, val)
                        result = cursor.fetchone()
                        if result:
                            student_name = result[0]

                    # Check if the student is recognized and mark attendance accordingly
                    if best_match_id:
                        self.mark_attendance(best_match_id)
                        if student_name:
                            attendance_status = f"Attendance taken for student {student_name}"
                        else:
                            attendance_status = "Attendance taken for student"
                    else:
                        attendance_status = "Attendance not taken"

                    # Draw rectangle and label with attendance status and student's name
                    color = (0, 255, 0) if best_match_id else (0, 0, 255)
                    cv2.rectangle(img, (x, y), (x+w, y+h), color, 2)
                    cv2.putText(img, attendance_status, (x, y-10), cv2.FONT_ITALIC, 0.9, color, 2)
                    if best_match_id and student_name:
                        cv2.putText(img, student_name, (x, y+h+20), cv2.FONT_ITALIC, 0.9, color, 2)

            cv2.imshow("Faces with Rectangles", img)

            if cv2.waitKey(1) == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()

    def limit_foldersize(self, num_of_images=100):
        db_folder = 'db'
        student_folders = [os.path.join(db_folder, folder) for folder in os.listdir(db_folder) if os.path.isdir(os.path.join(db_folder, folder))]
        
        for student_folder in student_folders:
            image_files = [os.path.join(student_folder, file) for file in os.listdir(student_folder) if file.lower().endswith(('.png', '.jpg', '.jpeg'))]
            num_images_to_remove = len(image_files) - num_of_images
            if num_images_to_remove > 0:
                # Sort the image files by modification time and keep only the most recent ones
                image_files.sort(key=lambda x: os.path.getmtime(x), reverse=True)
                images_to_remove = image_files[num_of_images:]
                for image in images_to_remove:
                    os.remove(image)


In [1]:
import cv2
import mysql.connector
from datetime import datetime
from deepface import DeepFace
import os
import pickle
import uuid
class Recognition:
    def __init__(self, db, session_id, course_id):
        self.db = db
        self.session_id = session_id
        self.course_id = course_id
        self.confidence_threshold = 0.6
        self.load_encodings()

    def load_encodings(self):
        # Load saved embeddings and file paths
        if os.path.exists('known_encodings.pkl'):
            with open('known_encodings.pkl', 'rb') as f:
                self.results = pickle.load(f)
                self.known_encodings = [result[1] for result in self.results]
                self.file_paths = [result[0] for result in self.results]
        else:
            self.results = []
            self.known_encodings = []
            self.file_paths = []

    def retrain(self):
        def process_folder(student_folder):
            student_id = os.path.basename(student_folder)
            image_files = [os.path.join(student_folder, file) for file in os.listdir(student_folder) if file.lower().endswith(('.png', '.jpg', '.jpeg'))]
            encodings = []
            for img_path in image_files:
                img = cv2.imread(img_path)
                encoding = DeepFace.represent(img, model_name='VGG-Face', enforce_detection=False)
                if encoding:
                    encodings.append((img_path, encoding[0]['embedding']))
            return encodings

        db_folder = 'db'
        student_folders = [os.path.join(db_folder, folder) for folder in os.listdir(db_folder) if os.path.isdir(os.path.join(db_folder, folder))]

        self.results = []
        for student_folder in student_folders:
            self.results.extend(process_folder(student_folder))

        # Save computed embeddings and corresponding file paths
        with open('known_encodings.pkl', 'wb') as f:
            pickle.dump(self.results, f)

        self.load_encodings()

    def mark_attendance(self, student_id):
        """Marks attendance for a recognized student in the database."""
        try:
            cursor = self.db.cursor()
            # Fetch student's name from the database based on the student_id
            sql = "SELECT fullname FROM student_details WHERE student_id = %s"
            val = (student_id,)
            cursor.execute(sql, val)
            result = cursor.fetchone()
            if result:
                student_name = result[0]
                attendance_status = f"Attendance taken for student {student_name}"
            else:
                attendance_status = "Attendance not taken"
            # Check if the student has already been marked present for the given course and session
            sql = "SELECT * FROM attendance WHERE student_id = %s AND course_id = %s AND session_id = %s"
            val = (student_id, self.course_id, self.session_id)
            cursor.execute(sql, val)
            result = cursor.fetchone()
            if result:
                print(f"Student with ID {student_id} has already been marked present for course {self.course_id} and session {self.session_id}. Skipping insertion.")
                return
            else:
                sql = "INSERT INTO attendance (course_id, session_id, timestamp, status, student_id) VALUES (%s, %s, %s, %s, %s)"
                val = (self.course_id, self.session_id, datetime.now(), "Present", student_id)
                cursor.execute(sql, val)
                self.db.commit()
                cursor.close()
                print(f"Attendance marked for student with ID {student_id} for course {self.course_id} and session {self.session_id}.")
        except mysql.connector.Error as err:
            print(f"Database error: {err}")
    def recognize(self, max_images=1000):
        cap = cv2.VideoCapture(0)
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        num_of_images = max_images
        while True:
            success, img = cap.read()
            if not success:
                print("Error reading frame from camera.")
                break
    
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
            for (x, y, w, h) in faces:
                face_img = img[y:y+h, x:x+w]
    
                face_encoding = DeepFace.represent(face_img, model_name='VGG-Face', enforce_detection=False)
    
                if face_encoding:
                    face_encoding = face_encoding[0]['embedding']
    
                    best_match_id = None
                    min_distance = float('inf')
    
                    for j, known_encoding in enumerate(self.known_encodings):
                        if len(known_encoding) == len(face_encoding):
                            result = DeepFace.verify(known_encoding, face_encoding, model_name='VGG-Face', silent=True)
                            if result['verified'] or result['distance'] > self.confidence_threshold:
                                # Extract the directory name (folder name) from the image file path
                                best_match_id = os.path.basename(os.path.dirname(self.file_paths[j]))
                
                    # Fetch student's name from the database based on student ID
                    student_name = None
                    if best_match_id:
                        cursor = self.db.cursor()
                        sql = "SELECT fullname FROM student_details WHERE student_id = %s"
                        val = (best_match_id,)
                        cursor.execute(sql, val)
                        result = cursor.fetchone()
                        if result:
                            student_name = result[0]

    
                    if best_match_id:
                        # Pass folder name as student_id
                        self.mark_attendance(best_match_id)
                        if student_name:
                            attendance_status = f"Attendance taken for student {student_name}"
                            # Save the successful recognition image to the student's folder
                            student_folder = os.path.join('db', best_match_id)
                            try:
                                os.makedirs(student_folder, exist_ok=True)  # Create the folder if it doesn't exist
                                image_files = [os.path.join(student_folder, file) for file in os.listdir(student_folder) if file.lower().endswith(('.png', '.jpg', '.jpeg'))]
                                num_images_in_folder = len(image_files)
                                if num_images_in_folder < num_of_images:  # Limit the number of images per student folder
                                    unique_filename = str(uuid.uuid4()) + '.jpg'
                                    cv2.imwrite(os.path.join(student_folder, unique_filename), face_img)
                                elif num_images_in_folder == num_of_images:
                                    print(f"Folder {student_folder} already contains {num_of_images} images. No new images will be saved.")
                                else:
                                    print(f"Folder {student_folder} contains more than {num_of_images} images. No new images will be saved.")
                            except Exception as e:
                                print(f"Error saving recognition image to folder {student_folder}: {e}")
    
                    else:
                        attendance_status = "Attendance not taken"
    
                    # Draw rectangle and label with attendance status and student's name
                    color = (0, 255, 0) if best_match_id else (0, 0, 255)
                    cv2.rectangle(img, (x, y), (x+w, y+h), color, 2)
                    cv2.putText(img, attendance_status, (x, y-10), cv2.FONT_ITALIC, 0.9, color, 2)
                    if best_match_id and student_name:
                        cv2.putText(img, student_name, (x, y+h+20), cv2.FONT_ITALIC, 0.9, color, 2)
    
            cv2.imshow("Faces with Rectangles", img)
    
            if cv2.waitKey(1) == ord('q'):
                break
    
        cap.release()
        cv2.destroyAllWindows()


In [2]:
session_id = 3
course_id = 2
# Example usage:
db_smp = mysql.connector.connect(
        host="127.0.0.1",
        user="root",
        password="e@123321xzX",
        port=3306,
        database="attendance"
    )

recognition = Recognition(db_smp, session_id, course_id)
recognition.retrain()
recognition.recognize(6)

Student with ID 8 has already been marked present for course 2 and session 3. Skipping insertion.
Student with ID 8 has already been marked present for course 2 and session 3. Skipping insertion.
Student with ID 8 has already been marked present for course 2 and session 3. Skipping insertion.
Student with ID 8 has already been marked present for course 2 and session 3. Skipping insertion.
Folder db\8 already contains 6 images. No new images will be saved.
Student with ID 8 has already been marked present for course 2 and session 3. Skipping insertion.
Folder db\8 already contains 6 images. No new images will be saved.
Student with ID 8 has already been marked present for course 2 and session 3. Skipping insertion.
Folder db\8 already contains 6 images. No new images will be saved.
Student with ID 8 has already been marked present for course 2 and session 3. Skipping insertion.
Folder db\8 already contains 6 images. No new images will be saved.
Student with ID 8 has already been marked 